First we need to import X_dataset.csv and provide labels to the dataset

In [1]:
import pyspark
import pandas as pd

headers = ["class", "largestSpot", "spotDistribution", "activity", "evolution", "previousActivity", "complex",
           "complexOnPath", "area", "largestSpotArea", "c-class", "m-class", "x-class"]

df = pd.read_csv("X_dataset.csv", header=None, names=headers)

def export_to_csv(df, filename):
    df.to_csv(filename, index=False)

In [ ]:
df = pd.read_csv("out.csv")


def make_correlation_report(df, filename):
    export_to_csv(df.corr(), filename)


# Correlation report for all columns
make_correlation_report(df, "correlation_analysis.csv")

# Correlation report for x-class flares
x_class = df.drop(["c-class", "m-class"], axis=1)
make_correlation_report(x_class, "xclass_correlation_analysis.csv")